In [73]:
import pandas as pd
import numpy as np
import random
#from catboost import CatBoostRanker, Pool, MetricVisualizer
#from copy import deepcopy

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [74]:
HotelSearch_train=pd.read_csv("./data/hotelsearch_training_predictions.csv", index_col=0)
HotelSearch_test=pd.read_csv("./data/hotelsearch_test_predictions.csv", index_col=0)

HotelSearch_train["PredictedRelevancy"]=round(HotelSearch_train["PredictedRelevancy"],4)
HotelSearch_test["PredictedRelevancy"]=round(HotelSearch_test["PredictedRelevancy"],4)
HotelSearch_train.head()


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,weekday,week_of_year,hour,minute,time_epoch,early_night,nans_count,ActualRelevancy,PredictedRelevancy,Predictedrank
0,71,24,216,NaN,NaN,98,76541,3,3.5,0,2.20,0.0482,3.21,1,16.90,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11,23,14,1.363130e+09,True,24,0.0,0.0435,1.0
1,71,24,216,NaN,NaN,98,37933,3,4.0,0,2.08,0.0812,3.59,2,18.34,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,1.0,0.0,46.0,NaN,NaN,NaN,0.0,0.0,NaN,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11,23,14,1.363130e+09,True,20,0.0,0.0238,4.0
2,71,24,216,NaN,NaN,98,108588,4,4.0,0,0.00,0.0268,4.08,3,26.78,1,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,1.0,0.0,108.0,1.0,0.0,32.0,-1.0,0.0,9.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11,23,14,1.363130e+09,True,17,0.0,0.0279,2.0
3,71,24,216,NaN,NaN,98,28672,3,4.0,0,2.20,0.0151,3.35,4,21.48,0,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11,23,14,1.363130e+09,True,24,0.0,0.0076,9.0
4,71,24,216,NaN,NaN,98,25741,3,3.5,0,1.61,0.0359,4.02,6,19.47,1,341,1,18,1,1,1,1,NaN,NaN,1,NaN,NaN,NaN,0.0,0.0,NaN,1.0,0.0,10.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11,23,14,1.363130e+09,True,19,0.0,0.0042,13.0


In [75]:
def transpose(df):
    return pd.DataFrame(
        {"sequence": [df["ActualRelevancy"].tolist()] + [None] * (len(df) - 1)}, index=df.index
    )


HotelSearch_train = pd.concat([HotelSearch_train, HotelSearch_train.groupby("srch_id").apply(transpose)], axis=1)
HotelSearch_test = pd.concat([HotelSearch_test, HotelSearch_test.groupby("srch_id").apply(transpose)], axis=1)
HotelSearch_test.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,weekday,week_of_year,hour,minute,time_epoch,early_night,nans_count,ActualRelevancy,PredictedRelevancy,Predictedrank,sequence
0,246,14,100,NaN,NaN,219,11396,4,5.0,0,0.69,0.0551,5.36,1,159.64,0,18294,1,6,2,0,1,0,NaN,87.17,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10,9,55,1.362391e+09,False,27,1.0,0.1222,1.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,246,14,100,NaN,NaN,219,103885,2,4.0,1,1.10,0.1540,4.75,2,98.32,0,18294,1,6,2,0,1,0,NaN,93.87,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10,9,55,1.362391e+09,False,25,0.0,0.1179,2.0,None
2,246,14,100,NaN,NaN,219,61167,3,4.5,0,1.10,0.0314,4.87,3,101.24,0,18294,1,6,2,0,1,0,NaN,92.31,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10,9,55,1.362391e+09,False,25,0.0,0.0549,3.0,None
3,246,14,100,NaN,NaN,219,95490,2,3.5,1,1.10,0.0142,4.88,4,95.40,0,18294,1,6,2,0,1,0,NaN,89.28,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10,9,55,1.362391e+09,False,27,0.0,0.0136,6.0,None
4,246,14,100,NaN,NaN,219,127213,3,4.0,1,1.10,0.1652,4.95,6,119.73,0,18294,1,6,2,0,1,0,NaN,93.97,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,10,9,55,1.362391e+09,False,25,0.0,0.0466,4.0,None


In [76]:
HotelSearch_train[["srch_id","site_id","position","ActualRelevancy","PredictedRelevancy","Predictedrank","sequence"]][HotelSearch_train.srch_id ==71]

,srch_id,site_id,position,ActualRelevancy,PredictedRelevancy,Predictedrank,sequence
0,71,24,1,0.0,0.0435,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,71,24,2,0.0,0.0238,4.0,None
2,71,24,3,0.0,0.0279,2.0,None
3,71,24,4,0.0,0.0076,9.0,None
4,71,24,6,0.0,0.0042,13.0,None
5,71,24,7,0.0,0.0036,15.0,None
6,71,24,8,0.0,0.0048,12.0,None
7,71,24,9,0.0,0.0032,16.0,None
8,71,24,10,0.0,0.0024,17.0,None
9,71,24,12,0.0,-0.0068,28.0,None


In [77]:
HotelSearch_test[["srch_id","site_id","position","ActualRelevancy","PredictedRelevancy","Predictedrank","sequence"]][HotelSearch_test.srch_id ==246]

,srch_id,site_id,position,ActualRelevancy,PredictedRelevancy,Predictedrank,sequence
0,246,14,1,1.0,0.1222,1.0,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,246,14,2,0.0,0.1179,2.0,None
2,246,14,3,0.0,0.0549,3.0,None
3,246,14,4,0.0,0.0136,6.0,None
4,246,14,6,0.0,0.0466,4.0,None
5,246,14,7,0.0,-0.0021,12.0,None
6,246,14,8,0.0,0.0024,9.0,None
7,246,14,9,0.0,-0.0166,16.0,None
8,246,14,10,0.0,0.0049,8.0,None
9,246,14,12,0.0,-0.0026,13.0,None
